# CONFIGURACIÓN DEL CLIENTE Y SERVIDOR RESTCONF

### Características del servidor RESTCONF de pruebas

El servidor de pruebas de RESTCONF funciona en una máquina virtual (debido a las ventajas que esto ofrece) con las siguientes características:

- SO: Ubuntu 16 64bits
- RAM: 3GB
- Almacenamiento: 12GB

Se hospeda a través del programa Virtual Box 6.0.

La instalación del SO se realiza sin ningún tipo de configuración avanzada.
#### Configuración de adicional de Virtual Box

- En la configuración de red de la máquina virtual, es necesario activar "Adaptador puente" para que la máquina virtual esté conectada a la misma red que la máquina anfitrión.
- Instalar las "Guest additions" de Virtual Box a la máquina virtual para poder compartir archivos y el portapapeles.
- Activar dos carpetas compartidas en /home, llamadas Shared y Backend, una que apunte a los archivos a compartir entre el cliente (Cygwin) y la máquina virtual y el otro que apunte a la carpeta donde se aloje el Backend en el repositorio de la máquina anfitrión.
- Enlazar las dos carpetas compartidas en Ubuntu (esto hay que hacerlo cada vez que se reinicia la máquina), a través de los comandos siguientes:
    - _sudo mount -t vboxsf Shared ./Shared_
    - _sudo mount -t vboxsf Backend ./Backend_

**Nota:** Esos comandos se ejecutan en una consola cuya ruta sea el directorio en donde se crearon las carpetas a enlazar.



### Configuración del servidor:

Una vez instalado el sistema operativo en el disco virtual, es necesario instalar el manejador de paquetes de Python (Pip) y Git.

_sudo apt-get install python3-pip_

_sudo apt-get install git_


#### Instalación del protocolo RESTCONF para Python

Existe una implementación del protocolo RESTCONF para Python llamada Jetconf, esa es la implementación que usa el servidor.

_python3 -m pip install jetconf_

Luego, el backend a utilizar será el recomendado por defecto para Jetconf: Jukebox

_git clone https://gitlab.labs.nic.cz/jetconf/jetconf-jukebox.git_

_cd jetconf-jukebox_

_sudo python3 setup.py install_

**A partir de acá, se siguen los pasos listados de la siguiente fuente:**
https://gitlab.labs.nic.cz/jetconf/jetconf-jukebox/wikis/setup

No es necesario que la conexión sea segura para poder realizar pruebas de conexión. Para deshabilitar la seguridad y exigencia de certificados, en el archivo de configuración (en formato yaml) del backend, en la sección de HTTP_SERVER, se establece como **true** los siguientes atributos:

- DBG_DISABLE_CERT: true
- DBG_DISABLE_CERTS: true
- DISABLE_SSL: true

En caso de querer que la conexión sea segura, es necesario crear unos certificados de seguridad, siguiendo los pasos presentados en la siguiente fuente: https://gitlab.labs.nic.cz/labs/jetconf/wikis/certificates

Los archivos necesarios se encuentran en la carpeta Utils del repositorio de Jetconf




#### Problemas presentados:
- **Error de sintaxis en la librería Yangson:** Instalar la versión 1.3.32. Ocurre por la versión de Python3 que instala Ubuntu (3.5.), por lo tanto, también se puede solucionar instalando una versión más reciente de Python.

### Configuración del Configuración

Para conectar al servidor se usará Curl como cliente, desde la consola de Cygwin en la máquina anfitrión, y sin seguridad en la conexión (por lo tanto no es necesario realizar el paso anterior).

Lo primero es encender el servidor de Jetconf, para ello es necesario dirigirse a la carpeta del backend, en este caso jetconf-jukebox, e iniciar el servidor mediante el siguiente comando:

_jetconf -c [NOMBRE-DEL-ARCHIVO].yaml_

El nombre por defecto del archivo, si se siguieron los pasos de instalación anteriores sin ninguna modificación, es config.

Posteriormente, desde la máquina anfitrión se inicia una consola Cygwin, y se procederá a probar la conexión mediante el siguiente comando:

_curl --http2-prior-knowledge -X GET http://[DIR-IP]:[PUERTO]/restconf/data_

Donde la dirección IP es la de la máquina virtual y el puerto es el mostrado en la consola desde donde se inició el servidor (por defecto es el 8443).

Si el servidor está funcionando correctamente, desde la consola de Cygwin se podrá ver la respuesta del servidor, siendo esta una estructura de datos, en formato Json, de los modelos YANG que incluía por defecto el backend.

#### Problemas presentados:
- **El Curl instalado en Cygwin no soporta http2:** Desde el instalador de Cygwin, agregar las librerías nghttp2 libnghttp2-dev y actualizar Curl (seleccionándolo también desde el instalador)